In [1]:
import os
import numpy as np
import time
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib as mpl
from matplotlib.lines import Line2D

mpl.use('Qt5Agg') 
plt.ion()  

plt.rcParams['font.sans-serif'] = ['Times New Roman', 'SimHei']
plt.rcParams['mathtext.fontset'] = 'cm'       
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
#    The main takes the following arguments:
#    Network_Type: Type of network (e.g., ER, SF, BA, ...)
#    N: Number of nodes
#    k_Pairs_teo: Average degree of the 1-hyperedges
#    k_Delta_teo: Average degree of the 2-hyperedges
#    Overlapness: Intra-order hyperedge overlap of the 2-hyperedges
#    Iter_burn: Number of iterations for the burn-in time  
#    Iter: Number of iterations for the observable computation
#    h: Integration step
#    sigma_T: Coupling strength of the 2-hyperedges 
#    sigma_P: Coupling strength of the 1-hyperedges
#    beta:Phase lag parameter
#    repeat: The average number of statistics
#    direction: Scanning direction: f (forward) or b (backward)

In [2]:

start_time = time.time()

exe_path = "HO-SK.exe"
network_type = "Regular_Hg"
N = 500
k_Pairs_teo = 5
k_Delta_teo = 6
Overlapness = 0.8
Iter_burn = 8000
Iter = 10000
h = 0.01
sigma_T = 4.0
sigma_P = 1.0     
phase_factor = 1.0
freq_factor = 1.0
beta = 0.2
use_init = 0     
direction = "f"  

cmd = (
    f"{exe_path} {network_type} {N} {k_Pairs_teo} {k_Delta_teo} {Overlapness} "
    f"{Iter_burn} {Iter} {h} {sigma_T} {sigma_P} "
    f"{phase_factor} {freq_factor} {beta} {use_init} {direction}"
)

print("Running command:")
print(cmd)
os.system(cmd)

results_dir = "Results"

base_args = (
    f"{direction}_pf_{phase_factor:.4f}_lP_{sigma_P:.4f}_lT_{sigma_T:.4f}_"
    f"{network_type}_N_{N}_k_{k_Pairs_teo}_kT_{k_Delta_teo}_"
    f"T_{Overlapness:.4f}_Iter_burn_{Iter_burn}_Iter_{Iter}_beta_{beta:.4f}"
)

r_file = f"{results_dir}/Stationary_{base_args}.txt"

print("\nReading result from:")
print(r_file)

with open(r_file, "r") as f:
    parts = f.readline().strip().split()
    _, r_val, rlink_val = map(float, parts)

print("\n===== RESULT =====")
print(f"R      = {r_val}")
print(f"R_link = {rlink_val}")

end_time = time.time()
print(f"\n⏱ Total runtime: {(end_time - start_time):.2f} sec")


Running command:
HO-SK.exe Regular_Hg 500 5 6 0.8 8000 10000 0.01 4.0 1.0 1.0 1.0 0.2 0 f

Reading result from:
Results/Stationary_f_pf_1.0000_lP_1.0000_lT_4.0000_Regular_Hg_N_500_k_5_kT_6_T_0.8000_Iter_burn_8000_Iter_10000_beta_0.2000.txt

===== RESULT =====
R      = 0.1128
R_link = 0.7531

⏱ Total runtime: 8.11 sec


In [ ]:
start_time = time.time()

exe_path = "HO-SK.exe"
network_type = "Regular_Hg"
N = 500
k_Pairs_teo = 5
k_Delta_teo = 6
Overlapness = 0.8
Iter_burn = 8000
Iter = 10000
h = 0.01
sigma_T = 4.0
phase_factor = 1.0
freq_factor = 1.0
beta = 0.2

sigma_P_list = [round(s, 2) for s in np.arange(0, 10.5, 0.5)]
repeat = 100      

results_dir = "Results"
os.makedirs(results_dir, exist_ok=True)

def run_scan(direction):

    sigmas = sigma_P_list if direction == "f" else list(reversed(sigma_P_list))
    n_sigma = len(sigmas)

    r_sum = np.zeros(n_sigma)
    rlink_sum = np.zeros(n_sigma)

    for rep in range(repeat):
        print(f"\n===== repetition {rep+1}/{repeat} of direction {direction} =====")

        for idx, sigma_P in enumerate(sigmas):

            use_init = 0 if idx == 0 else 1   

            print(f"σ_P = {sigma_P:.2f}, use_init = {use_init}")

            cmd = (
                f"{exe_path} {network_type} {N} {k_Pairs_teo} {k_Delta_teo} {Overlapness} "
                f"{Iter_burn} {Iter} {h} {sigma_T} {sigma_P} "
                f"{phase_factor} {freq_factor} {beta} {use_init} {direction}"
            )
            os.system(cmd)

            base_args = (
                f"{direction}_pf_{phase_factor:.4f}_lP_{sigma_P:.4f}_lT_{sigma_T:.4f}_"
                f"{network_type}_N_{N}_k_{k_Pairs_teo}_kT_{k_Delta_teo}_"
                f"T_{Overlapness:.4f}_Iter_burn_{Iter_burn}_Iter_{Iter}_beta_{beta:.4f}"
            )

            r_file = f"{results_dir}/Stationary_{base_args}.txt"

            with open(r_file, "r") as f:
                parts = f.readline().strip().split()
                _, r_val, rlink_val = map(float, parts)

            r_sum[idx] += r_val
            rlink_sum[idx] += rlink_val

    r_avg = r_sum / repeat
    rlink_avg = rlink_sum / repeat

    if direction == "b":
        return sigma_P_list, r_avg[::-1], rlink_avg[::-1]
    else:
        return sigma_P_list, r_avg, rlink_avg

sigma_f, r_f, rlink_f = run_scan("f")
sigma_b, r_b, rlink_b = run_scan("b")

output_f = (
    f"{results_dir}/Averaged_Results_f"
    f"_N_{N}_T_{Overlapness:.4f}_beta_{beta:.4f}"
    "_sigma_r_rlink.txt"
)

output_b = (
    f"{results_dir}/Averaged_Results_b"
    f"_N_{N}_T_{Overlapness:.4f}_beta_{beta:.4f}"
    "_sigma_r_rlink.txt"
)

with open(output_f, "w") as f:
    for s, r, rl in zip(sigma_f, r_f, rlink_f):
        f.write(f"{s:.4f}\t{r:.4f}\t{rl:.4f}\n")

with open(output_b, "w") as f:
    for s, r, rl in zip(sigma_b, r_b, rlink_b):
        f.write(f"{s:.4f}\t{r:.4f}\t{rl:.4f}\n")

print("\n==================== Saved successfully ====================")
print("f:", output_f)
print("b:", output_b)

end_time = time.time()
print(f"\n⏱ Total runtime: {(end_time - start_time)/60:.2f} min")


In [ ]:
file_f = "Results/Averaged_Results_f_N_500_T_0.8000_beta_0.2000_sigma_r_rlink.txt"
file_b = "Results/Averaged_Results_b_N_500_T_0.8000_beta_0.2000_sigma_r_rlink.txt"

data_f = np.loadtxt(file_f)
data_b = np.loadtxt(file_b)

sigma_f, r_f, rlink_f = data_f[:, 0], data_f[:, 1], data_f[:, 2]
sigma_b, r_b, rlink_b = data_b[:, 0], data_b[:, 1], data_b[:, 2]

plt.figure(figsize=(5, 3))

plt.plot(sigma_f, r_f, "-o", color="blue", markersize=4,
         label="forward scan (f)")

plt.plot(sigma_b, r_b, "-s", color="red", markersize=4,
         label="backward scan (b)")

plt.xlabel(r"$\gamma_1$", fontsize=16)
plt.ylabel(r"$R$", fontsize=16)

plt.xlim(sigma_f.min(), sigma_f.max())
plt.ylim(0, 1.1)

plt.grid(False)
plt.legend(fontsize=12)
plt.tight_layout()
plt.legend().remove()
plt.show()
